In [3]:
import pandas as pd

In [4]:
anime = pd.read_csv('./raw/anime-dataset-2023.csv')

FileNotFoundError: [Errno 2] No such file or directory: './raw/anime-dataset-2023.csv'

In [ ]:
anime

In [ ]:
df = pd.read_csv('users-details-2023.csv')

In [ ]:
df.head()

In [ ]:
users = df[['Mal ID', 'Username', 'Mean Score', 'Total Entries']].copy()

In [ ]:
from tqdm import tqdm
chunk_size = 5 * 10 ** 6
chunks = []

for i, chunk in enumerate(tqdm(pd.read_csv('users-score-2023.csv', chunksize=chunk_size))):
    processed = chunk[['user_id', 'anime_id', 'rating']]
    processed.to_parquet(f'score/users_scores_{i}.parquet', index=False)


In [ ]:
score = pd.read_parquet('score/users_scores_4.parquet')

In [ ]:
score.head()

In [ ]:
anime = pd.read_csv('anime-dataset-2023.csv')
anime.shape

In [ ]:
anime

In [ ]:
users

In [ ]:
users['Ratings'] = 0

In [ ]:
users

In [ ]:
rating = pd.read_parquet(f'score/users_scores_0.parquet')

In [ ]:
counter = rating.groupby("user_id").agg(count=('user_id', 'count')).reset_index()
counter.info()

In [ ]:
users['Ratings']

In [ ]:
users = df[['Mal ID', 'Username', 'Mean Score', 'Total Entries']].copy()
rating = pd.read_parquet(f'score/users_scores_4.parquet')
counter = rating.groupby("user_id").agg(count=('user_id', 'count')).reset_index()

In [ ]:
merged = users.merge(counter, how='left', left_on='Mal ID', right_on='user_id')

In [ ]:
merged

In [ ]:
users = df[['Mal ID', 'Username', 'Mean Score', 'Total Entries']].copy() 

for i in range(5):
    rating = pd.read_parquet(f'score/users_scores_{i}.parquet')
    grouped_rating = rating.groupby("user_id").agg(count=('user_id', 'count')).rename({'count': f'count_{i}'}, axis=1).reset_index()
    users = users.merge(grouped_rating, how='left', left_on='Mal ID', right_on='user_id').drop('user_id', axis=1)

In [ ]:
users["Ratings"] = users[['count_0', 'count_1', 'count_2', 'count_3', 'count_4']].sum(axis=1)

In [ ]:
users.drop(['count_0', 'count_1', 'count_2', 'count_3', 'count_4'], axis=1, inplace=True)

In [ ]:
users['Ratings'] = users['Ratings'].astype('int16')

In [ ]:
users

In [ ]:
users['Total Entries'].isna().sum()
users['Total Entries'].fillna(0, inplace=True)

In [ ]:
users['Total Entries'].isna().sum()

In [ ]:
users['Total Entries'] = users['Total Entries'].astype('int16')

In [ ]:
users['Mean Score'].fillna(0, inplace=True)

In [ ]:
users['Username'].fillna('Unknown', inplace=True)

In [ ]:
users.to_parquet('users/users.parquet', index=False)

In [ ]:
test = pd.read_parquet('users/users.parquet')

In [ ]:
anime = pd.read_csv('raw/anime-dataset-2023.csv')
anime.shape

In [ ]:
anime

In [ ]:
essential_cols = ['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres', 'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status', 'Studios', 'Source', 'Duration', 'Rating', 'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL']
filtered = anime[essential_cols].copy()
filtered

In [ ]:
import numpy as np

filtered['Score'] = pd.to_numeric(filtered['Score'].replace('UNKNOWN', np.nan), errors='coerce')
filtered['Score'].fillna(0.0, inplace=True)
filtered['Score'].astype(float)

In [ ]:
filtered['Scored By'] = pd.to_numeric(filtered['Scored By'].replace('UNKNOWN', np.nan), errors='coerce')
filtered['Scored By'].fillna(0.0)

In [ ]:
filtered['Scored By'] = filtered['Scored By'].fillna(0.0)

In [ ]:
filtered['Scored By'].isna().sum()

In [ ]:
filtered['Scored By'] = filtered['Scored By'].astype('int64')

In [ ]:
filtered.info()

In [ ]:
filtered

In [ ]:
filtered['Mod_name'] = filtered['Name'].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
filtered['Mod_name'] = filtered['Mod_name'].str.lower()
filtered['Mod_name'] = filtered['Mod_name'].str.replace("\s+", ' ', regex=True)
filtered = filtered[filtered["Mod_name"].str.len() > 0]

In [ ]:
filtered

In [ ]:
filtered.to_parquet('anime/anime.parquet', index=False)

In [ ]:
%timeit
old = pd.read_parquet('anime/anime.parquet')

In [ ]:
filtered.loc[55]